In [ ]:
import gdown

In [ ]:
!gdown --id 1KmoZYmPzxrrtR4a3mSfPIumbDwf_Mrpw > /dev/null 2>&1
!gdown --id 19vK08esg9hQtD8JB-9jm7Qkck7nZ-ZqR > /dev/null 2>&1

## About the Dataset
This a YouTube comments dataset scrapped using the [YouTube Data API](https://developers.google.com/youtube/v3/docs/comments). Currently the dataset consists of roughly **1.12M** comments. The dataset can be accessed [here](https://drive.google.com/drive/folders/1-9OnYbFuiSA0M7skOYche5erzZ4Ba5Nz?usp=sharing).
<br><br>
The dataset currently has text in _English, Hindi and Hinglish_.
Please refer to the following information for an overview of the columns and the corresponding data stored within them. [Link](https://github.com/aatmanvaidya/Sentiment-Analysis-of-Online-Harassment-Towards-Women-Wrestlers/blob/scraper/attributes.txt).

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
from collections import Counter
import re

In [ ]:
%%time
df = pd.read_feather(r'comments_cleaned_feather.feather')

CPU times: user 1.43 s, sys: 907 ms, total: 2.34 s
Wall time: 1.98 s


In [ ]:
df.head()

,videoId,textDisplay,textOriginal,authorDisplayName,authorProfileImageUrl,authorChannelUrl,authorChannelId,canRate,viewerRating,likeCount,publishedAt,updatedAt,parentId,commentId
0,EBOKYsWUhvI,Dub maro jaato 😢😢.<br>Tumse tumare ladkiya nhi...,Dub maro jaato 😢😢.\nTumse tumare ladkiya nhi b...,HECTOR OF TROY,https://yt3.ggpht.com/ytc/AOPolaQNP5bd7gNvbAas...,http://www.youtube.com/channel/UC5G8fjqoiFIqHp...,{'value': 'UC5G8fjqoiFIqHpKyVeOTsFg'},True,none,0,2023-07-06T07:04:02Z,2023-07-06T07:04:02Z,None,UgwbyvIkkAhUdaCFpcp4AaABAg
1,EBOKYsWUhvI,Haar gye bechare,Haar gye bechare,Geeta Saini,https://yt3.ggpht.com/8XmOPNrT3Vy3wr0fItlWbaMk...,http://www.youtube.com/channel/UCsMGRdH3YHrbs2...,{'value': 'UCsMGRdH3YHrbs21NOraRuyQ'},True,none,0,2023-07-03T22:21:37Z,2023-07-03T22:21:37Z,None,Ugyz3OwSXamho91-8I94AaABAg
2,EBOKYsWUhvI,Dhamki mili pahalwano ko aur sab manage kr liy...,Dhamki mili pahalwano ko aur sab manage kr liy...,Ayaan Chouhan,https://yt3.ggpht.com/ytc/AOPolaREH2WnrnbD53OI...,http://www.youtube.com/channel/UC_dWuNh6zydTHI...,{'value': 'UC_dWuNh6zydTHIRr6hi3Omg'},True,none,0,2023-07-03T05:06:33Z,2023-07-03T05:06:33Z,None,Ugyphs1TT1Yoj7MZBVJ4AaABAg
3,EBOKYsWUhvI,Jaato pr ye boj rhega ki vo apni hi vyavstha s...,Jaato pr ye boj rhega ki vo apni hi vyavstha s...,Kamal,https://yt3.ggpht.com/ytc/AOPolaTRSm_dEOKj9H82...,http://www.youtube.com/channel/UCVdGObpHM-IMHB...,{'value': 'UCVdGObpHM-IMHB_b7_K-7rA'},True,none,1,2023-07-02T03:39:50Z,2023-07-02T03:39:50Z,None,UgwnIxpuFAcKsEzffBp4AaABAg
4,EBOKYsWUhvI,Pahlwan jante h kuch nhi kr payenge uska,Pahlwan jante h kuch nhi kr payenge uska,jagriti tiwari upp,https://yt3.ggpht.com/ytc/AOPolaRrFnzD2i3N_3rk...,http://www.youtube.com/channel/UCRkcewHFhxE5Kf...,{'value': 'UCRkcewHFhxE5KfLHrLx0wpA'},True,none,0,2023-06-28T15:10:36Z,2023-06-28T15:10:36Z,None,UgxjuOlDigsmuDu53J54AaABAg


In [ ]:
df.shape

(1119948, 14)

In [ ]:
df['textOriginal'] = df['textOriginal'].astype(str)

In [ ]:
duplicate_rows = df[df.duplicated(subset='textOriginal', keep=False)]

In [ ]:
len(df['textDisplay'].unique())

93475

In [ ]:
# Now this is strange, the dataset doesnt have duplicate or empty rows, why is this happening? is it some datatype problem?
# Lets try to do a simple check for it again

In [ ]:
%%time
count = 0
value_list = []
for index, row in df.iterrows():
    value_list.append(row['textOriginal'])
    count+=1
print(count)
print(len(value_list))

1119948
1119948
CPU times: user 55 s, sys: 143 ms, total: 55.1 s
Wall time: 59.9 s


In [ ]:
# This means we are penetrating through the entire dataset. I dont know why pandas is doing that ://
# There is one small problem, there is a \u200b character that is present, we might have to remove that.

## Simple Slur List Frequency Count
This is a simple frequency count of how many words from the slur list are found in the data.
I have stored the results in a dictonary called _'slur_counts'_

In [ ]:
with open('slur_list.txt', 'r') as file:
    slur_words = [word.strip() for word in file.readlines()]

In [ ]:
len(slur_words)

506

In [ ]:
slur_words_set = set(slur_words)

In [ ]:
# A dictonary to count the frequency of each slur word identified
slur_counts = {}

In [ ]:
'''
for text in df['textOriginal']:
    words = text.split()
    for word in words:
        if word in slur_words:
            if word in slur_counts:
                slur_counts[word] += 1
            else:
                slur_counts[word] = 1
'''
for text in df['textOriginal']:
    if text is not None:
        words = text.split()
        slur_words_in_text = set(words) & slur_words_set
        for word in slur_words_in_text:
            if word in slur_counts:
                slur_counts[word] += 1
            else:
                slur_counts[word] = 1
# Problem with code, in set if a user has used the same word multiple times, set will dismiss that.

In [ ]:
sorted_slur_counts = dict(sorted(slur_counts.items(), key=lambda item: item[1], reverse=True))

In [ ]:
sorted_slur_counts
# list(sorted_slur_counts.items())[:8]

{'chutiya': 2647,
 'chutiye': 1839,
 'chod': 1788,
 'mc': 1155,
 'bsdk': 1147,
 'stupid': 994,
 'sali': 992,
 'gaddar': 981,
 'harami': 965,
 'बलात्कार': 790,
 'गद्दार': 623,
 'tatti': 499,
 'chamcha': 488,
 'fat': 482,
 'Randi': 418,
 'balatkar': 395,
 'jihadi': 383,
 'साली': 355,
 'chuda': 329,
 'हरामी': 315,
 'आतंकवादी': 295,
 'Madarchod': 270,
 'slave': 267,
 'saali': 252,
 'bai': 246,
 'halala': 245,
 'aatankwadi': 232,
 'chatukar': 228,
 'chamchi': 213,
 'ugly': 210,
 'chutia': 206,
 'चाटुकार': 188,
 '---': 157,
 'fuck': 155,
 'bhadva': 153,
 'bastard': 152,
 'Gandu': 150,
 'Gaand': 147,
 'bhenchod': 142,
 'गांड': 135,
 'mooh': 115,
 'lodu': 108,
 'burnol': 103,
 'मादरचोद': 101,
 'चमची': 99,
 'Gud': 97,
 'kutti': 93,
 'kutiya': 90,
 'सूअर': 90,
 'Maderchod': 89,
 'nasty': 83,
 'librandu': 83,
 'jihadis': 80,
 'चमचा': 76,
 'रंडी': 72,
 'दोगली': 69,
 'Gatiya': 67,
 'chakka': 67,
 'Lodu': 66,
 'चूतिये': 65,
 'chudai': 65,
 'Suar': 64,
 'जिहादी': 63,
 'bazaru': 61,
 'bhosdiwale': 59,

In [ ]:
sum(sorted_slur_counts.values())

25393

In [ ]:
'''
One thing is clear from this, all the explicit words that are in english are not there in the list,
this means youtube either deletes them or hides them. And the moderation for local vernacular language is weak.
'''

'\nOne thing is clear from this, all the explicit words that are in english are not there in the list,\nthis means youtube either deletes them or hides them. And the moderation for local vernacular language is weak.\n'

# Regex
What is a Regex? -> a sequence of characters that specifies a match pattern in text.

**Step 1** - Clean and Preprocess the Data
1.   Remove User Names - `r'@\w+\b'`
2.   Remove URL's and unecessary puntuation marks - `[^\w\s.]|http\S+|www\S+|https\S+`
3.  Remove Double spaces - `r'\s+', ' '`
4.  Remove any leading or trailing spaces - `.strip()`



In [ ]:
text = "@user, temp, temp, ;;;;;;;; I hate the offensive and racist comments\n\n\n. They disgust me. Visit https://example.com for more information."

In [ ]:
def clean_text(text):
    cleaned_text = re.sub(r'@\w+\b|[^\w\s.]|(?:https?|ftp)://\S+|www.\S+', '', text).replace('\n', '').strip()
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
    return cleaned_text

In [ ]:
print(clean_text(text))

temp temp I hate the offensive and racist comments. They disgust me. Visit for more information.


In [ ]:
regexDf = df[['textOriginal']]

In [ ]:
# clean text
regexDf['textOriginal'] = regexDf['textOriginal'].apply(lambda x: clean_text(x))

<ipython-input-26-547ff038857b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  regexDf['textOriginal'] = regexDf['textOriginal'].apply(lambda x: clean_text(x))


### Just check the count of words from the slur list ignoring case (upper or lower) of the word. We also look if any word from the slur list is present as a sub-string of any other word in the dataset.

1.   `re.findall()` - searches for all non-overlapping occurrences of a pattern in a given text.
2.   `(?i)` - make matching case-insenstive
3.  `'(' + '|'.join(slur_words) + r')'` - find a pattern that matches any of the slur words

In [ ]:
skip_regex = False

In [ ]:
%%time
slur_word_counts = {}
for index, row in regexDf.iterrows():
    text = row['textOriginal']
    # matches = re.findall(r"(?i)\b(" + '|'.join(slur_words) + r")\b", text)
    matches = re.findall(r'(?i)(' + '|'.join(slur_words) + r')', text)
    for match in matches:
        slur_word_counts[match] = slur_word_counts.get(match, 0) + 1

CPU times: user 12min 5s, sys: 1.66 s, total: 12min 7s
Wall time: 12min 12s


In [ ]:
# slur_word_counts

In [ ]:
sum(slur_word_counts.values())

101382

In [ ]:
sorted_slur_word_counts = dict(sorted(slur_word_counts.items(), key=lambda item: item[1], reverse=True))
print(list(sorted_slur_word_counts.items())[:8])

[('ms', 13407), ('bai', 9483), ('halwa', 8551), ('chod', 7929), ('chutiya', 3554), ('fat', 3144), ('mc', 3103), ('balatkar', 2631)]


In [ ]:
# We clearly see that the amount of slur words drastically increase. Now the step ahead would be too create seperate regex for the most used slur words.
# Note - the substrings could also be a part of different words and not necessarily hateful. But given the nature of the words, I feel that chances are low.

## Create Regex for the most used Slur Words

- Lets start by looking at the word `"chutiya"`

In [ ]:
text = "they called me a chhhutiyyaa and I ignored them."
# text = "abcchhutiyeexvy"

In [ ]:
pattern = r'ch+h?[uo]t[iy]a?'

In [ ]:
matches = re.findall(pattern, text, re.IGNORECASE)

In [ ]:
matches

['chhhuti']

In [ ]:
%%time
slur_word_count_2 = {}
for index, row in regexDf.iterrows():
    text = row['textOriginal']
    matches = re.findall(pattern, text, re.IGNORECASE)
    for match in matches:
        slur_word_count_2[match] = slur_word_count_2.get(match, 0) + 1

CPU times: user 58.8 s, sys: 154 ms, total: 58.9 s
Wall time: 59.5 s


In [ ]:
sum(slur_word_count_2.values())

9289

In [ ]:
sorted_slur_word_count_2 = dict(sorted(slur_word_count_2.items(), key=lambda item: item[1], reverse=True))
print(list(sorted_slur_word_count_2.items())[:8])

[('chuti', 6383), ('Chuti', 1168), ('choti', 463), ('chutia', 344), ('chuty', 204), ('chutya', 201), ('chhoti', 167), ('Chutya', 90)]


In [ ]:
# Okay wow, the number of words that are similar to `chutiya` increase drastically.

In [ ]:
# one problem with this is, we dont know what the sentence might be directed to, to women? to the govt? to brijbhusan?
# or just some guy reply to a person in a comment?

- now lets look at `chod`

In [ ]:
text_2 = "they called me a chhut, chod but I ignored them, hmpf"

In [ ]:
pattern_2 = r'ch+[ou]+[dt]'

In [ ]:
matches_2 = re.findall(pattern_2, text_2, re.IGNORECASE)

In [ ]:
matches_2

['chhut', 'chod']

In [ ]:
slur_word_count_3 = {}
for index, row in regexDf.iterrows():
    text = row['textOriginal']
    matches_2 = re.findall(pattern_2, text, re.IGNORECASE)
    for match in matches_2:
        slur_word_count_3[match] = slur_word_count_3.get(match, 0) + 1

In [ ]:
sum(slur_word_count_3.values())

27220

In [ ]:
sorted_slur_word_count_3 = dict(sorted(slur_word_count_3.items(), key=lambda item: item[1], reverse=True))
print(list(sorted_slur_word_count_3.items())[:8])

[('chod', 9750), ('chut', 8219), ('chot', 1640), ('chud', 1605), ('Chut', 1474), ('chhod', 1416), ('Choud', 382), ('chhot', 371)]


- now lets do this for `bsdk`

In [ ]:
text_3 = "they called me a bsssdk, I left"

In [ ]:
pattern_3 = r'bs+d+k'
matches = re.findall(pattern_3, text_3, re.IGNORECASE)

In [ ]:
matches

['bsssdk']

In [ ]:
slur_word_count_4 = {}
for index, row in regexDf.iterrows():
    text = row['textOriginal']
    matches = re.findall(pattern_3, text, re.IGNORECASE)
    for match in matches:
        slur_word_count_4[match] = slur_word_count_4.get(match, 0) + 1

In [ ]:
sum(slur_word_count_4.values())

1836

In [ ]:
sorted_slur_word_count_4 = dict(sorted(slur_word_count_4.items(), key=lambda item: item[1], reverse=True))
print(list(sorted_slur_word_count_4.items())[:8])

[('bsdk', 1323), ('Bsdk', 456), ('Bssdk', 32), ('BSDK', 25)]


- now lets try for a hindi word - `बलात्कार`

some links that help me understand this better
- https://stackoverflow.com/questions/41356013/how-to-detect-if-a-string-contains-hindi-devnagri-in-it-with-character-and-wor
- https://stackoverflow.com/questions/14859957/regular-expressions-with-indian-characters
- https://unicode.org/charts/PDF/U0900.pdf

## Lets now look at the impact regex has had on counting the slur words in the data

The slur list has 506 words consisting of different languages.

|        **Word**       | **Before Regex** |     **After Regex**     |
|:---------------------:|:----------------:|:-----------------------:|
| Simple Count of Words |      25,393      | 101,382 (just  english) |
|        chutiya        |       2,647      |          9,289          |
|          chod         |       1,788      |          12,749         |
|          bsdk         |       1,147      |          1,836          |


- what are the 100 senteces which use the slur words before and after regex?
- look at the slur count vs timeline

## Identify Text Language

In [ ]:
#supressing output
# !pip install inltk==0.9 > /dev/null 2>&1
# !pip install --upgrade fastai > /dev/null 2>&1

In [ ]:
!pip install polyglot==16.7.4 > /dev/null 2>&1
!pip install PyICU > /dev/null 2>&1
!pip install pycld2 > /dev/null 2>&1

In [ ]:
# Use iNLTK, Indic NLP Library or polyglot
# from inltk.inltk import identify_language, reset_language_identifying_models
from polyglot.detect import Detector
# languages supported by polyglot - https://polyglot.readthedocs.io/en/latest/Detection.html#supported-languages

In [ ]:
# text = 'आप कैसे हैं?' text = 'તમે કેમ છો' text = 'எப்படி இருக்கிறீர்கள்' text = 'സുഖമാണോ' text = 'तू कसा आहेस'
text_sample = 'आप कैसे हैं?'

In [ ]:
detector = Detector(text_sample)
print(detector.language)
print(detector.language.code)

name: Hindi       code: hi       confidence:  96.0 read bytes:  1638
hi


In [ ]:
langDf = df[['videoId','textOriginal','authorChannelId', 'likeCount', 'publishedAt', 'updatedAt', 'commentId']]

In [ ]:
%%time
langDf['textOriginal'] = langDf['textOriginal'].apply(lambda x: clean_text(x))

CPU times: user 21.6 s, sys: 390 ms, total: 21.9 s
Wall time: 25.4 s


<timed exec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
langDf['language'] = 'dnd' # dnd - did not detect

<ipython-input-62-94e6cd841e46>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  langDf['language'] = 'dnd' # dnd - did not detect


In [ ]:
# langDf.head()

In [ ]:
import warnings
# warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=UserWarning, module="polyglot")

In [ ]:
# %%time
# for index, row in langDf.iterrows():
#     text = row['textOriginal']
#     try:
#         detector = Detector(text)
#         detected_language = detector.language.code
#         langDf.at[index, 'language'] = detected_language
#     except:
#         # If an error occurs or language detection fails, keep the 'dnd' value
#         # An error could also occur if text length is small
#         pass

## Running the Seintiment Analysis Model

In [ ]:
!pip install transformers > /dev/null 2>&1
!pip install --upgrade accelerate > /dev/null 2>&1
# !pip install xformers > /dev/null 2>&1

In [ ]:
from transformers import pipeline
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax

In [ ]:
# https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment-latest
model_path = f"cardiffnlp/twitter-roberta-base-sentiment-latest"

In [ ]:
# sentiment_task = pipeline("sentiment-analysis", model=model_path, tokenizer=model_path)
# sentiment_task("I am very happy today")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_path)
config = AutoConfig.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
text = "Covid cases are increasing fast!"

In [ ]:
def apply_sentiment_analysis(text):
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = np.exp(scores) / np.exp(scores).sum()
    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    label = config.id2label[ranking[0]]
    score = scores[ranking[0]]
    return label, score

In [ ]:
label, score = apply_sentiment_analysis(text)
print(f"{label}, {score}")

negative, 0.7235763072967529


In [ ]:
sentimentDf = df[['videoId','textOriginal','authorChannelId', 'likeCount', 'publishedAt', 'updatedAt', 'commentId']]

In [ ]:
sentimentDf2 = sentimentDf.head(5000)

In [ ]:
sentimentDf2.shape

(5000, 7)

In [ ]:
sentimentDf2['label'] = ""
sentimentDf2['score'] = 0.0

<ipython-input-77-11ed32de50ef>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sentimentDf2['label'] = ""
<ipython-input-77-11ed32de50ef>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sentimentDf2['score'] = 0.0


In [ ]:
sentimentDf2.head()

,videoId,textOriginal,authorChannelId,likeCount,publishedAt,updatedAt,commentId,label,score
0,EBOKYsWUhvI,Dub maro jaato 😢😢.\nTumse tumare ladkiya nhi b...,{'value': 'UC5G8fjqoiFIqHpKyVeOTsFg'},0,2023-07-06T07:04:02Z,2023-07-06T07:04:02Z,UgwbyvIkkAhUdaCFpcp4AaABAg,,0.0
1,EBOKYsWUhvI,Haar gye bechare,{'value': 'UCsMGRdH3YHrbs21NOraRuyQ'},0,2023-07-03T22:21:37Z,2023-07-03T22:21:37Z,Ugyz3OwSXamho91-8I94AaABAg,,0.0
2,EBOKYsWUhvI,Dhamki mili pahalwano ko aur sab manage kr liy...,{'value': 'UC_dWuNh6zydTHIRr6hi3Omg'},0,2023-07-03T05:06:33Z,2023-07-03T05:06:33Z,Ugyphs1TT1Yoj7MZBVJ4AaABAg,,0.0
3,EBOKYsWUhvI,Jaato pr ye boj rhega ki vo apni hi vyavstha s...,{'value': 'UCVdGObpHM-IMHB_b7_K-7rA'},1,2023-07-02T03:39:50Z,2023-07-02T03:39:50Z,UgwnIxpuFAcKsEzffBp4AaABAg,,0.0
4,EBOKYsWUhvI,Pahlwan jante h kuch nhi kr payenge uska,{'value': 'UCRkcewHFhxE5KfLHrLx0wpA'},0,2023-06-28T15:10:36Z,2023-06-28T15:10:36Z,UgxjuOlDigsmuDu53J54AaABAg,,0.0


In [ ]:
%%time
# for index, row in sentimentDf2.iterrows():
#     text = row['textOriginal']
#     label, score = apply_sentiment_analysis(text)
#     row['label'] = label
#     row['score'] = score

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.39 µs


In [ ]:
sentimentDf2.head()

,videoId,textOriginal,authorChannelId,likeCount,publishedAt,updatedAt,commentId,label,score
0,EBOKYsWUhvI,Dub maro jaato 😢😢.\nTumse tumare ladkiya nhi b...,{'value': 'UC5G8fjqoiFIqHpKyVeOTsFg'},0,2023-07-06T07:04:02Z,2023-07-06T07:04:02Z,UgwbyvIkkAhUdaCFpcp4AaABAg,,0.0
1,EBOKYsWUhvI,Haar gye bechare,{'value': 'UCsMGRdH3YHrbs21NOraRuyQ'},0,2023-07-03T22:21:37Z,2023-07-03T22:21:37Z,Ugyz3OwSXamho91-8I94AaABAg,,0.0
2,EBOKYsWUhvI,Dhamki mili pahalwano ko aur sab manage kr liy...,{'value': 'UC_dWuNh6zydTHIRr6hi3Omg'},0,2023-07-03T05:06:33Z,2023-07-03T05:06:33Z,Ugyphs1TT1Yoj7MZBVJ4AaABAg,,0.0
3,EBOKYsWUhvI,Jaato pr ye boj rhega ki vo apni hi vyavstha s...,{'value': 'UCVdGObpHM-IMHB_b7_K-7rA'},1,2023-07-02T03:39:50Z,2023-07-02T03:39:50Z,UgwnIxpuFAcKsEzffBp4AaABAg,,0.0
4,EBOKYsWUhvI,Pahlwan jante h kuch nhi kr payenge uska,{'value': 'UCRkcewHFhxE5KfLHrLx0wpA'},0,2023-06-28T15:10:36Z,2023-06-28T15:10:36Z,UgxjuOlDigsmuDu53J54AaABAg,,0.0
